In [7]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr


# Load data
data = pd.read_csv('PLUG.csv', sep=',', engine='python')

assets = data.columns.values[1:].tolist()
data = data.iloc[:, 1:]

# Load index
index = pd.read_csv('TQQQ.csv', sep=',', engine='python')
index = index.iloc[-data.values.shape[0]:, 1:]


# Normalize data
scaler = MinMaxScaler([0.1,0.9])
data_X = scaler.fit_transform(data)
scaler_index = MinMaxScaler([0.1,0.9])
index = scaler_index.fit_transform(index)

# Number of components
N_COMPONENTS = 3

## Autoencoder - Keras
# Network hyperparameters
n_inputs = len(assets)
n_core = N_COMPONENTS
n_outputs = n_inputs

# Create model
input = Input(shape=(n_inputs,))
# Encoder
encoded = Dense(n_core, activation='sigmoid')(input)
# Decoder
decoded = Dense(n_outputs, activation='sigmoid')(encoded)

# define model
autoencoder = Model(input, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Testing in-sample
X_train = data_X
X_test = data_X

# Training parameters
epochs = 20

# Fit the model
history = autoencoder.fit(X_train,\
                          X_train,\
                          epochs=epochs,\
                          batch_size=1,\
                          shuffle=True,\
                          verbose=1)

# Make AE predictions
y_pred_AE_keras = autoencoder.predict(X_test)

print('test loss: '+str(autoencoder.evaluate(y_pred_AE_keras, X_test)))



Epoch 1/20
252/252 [==============================] - 0s 513us/step - loss: 0.1048
Epoch 2/20
252/252 [==============================] - 0s 495us/step - loss: 0.0606
Epoch 3/20
252/252 [==============================] - 0s 481us/step - loss: 0.0414
Epoch 4/20
252/252 [==============================] - 0s 446us/step - loss: 0.0333
Epoch 5/20
252/252 [==============================] - 0s 450us/step - loss: 0.0292
Epoch 6/20
252/252 [==============================] - 0s 513us/step - loss: 0.0268
Epoch 7/20
252/252 [==============================] - 0s 448us/step - loss: 0.0250
Epoch 8/20
252/252 [==============================] - 0s 469us/step - loss: 0.0236
Epoch 9/20
252/252 [==============================] - 0s 486us/step - loss: 0.0222
Epoch 10/20
252/252 [==============================] - 0s 442us/step - loss: 0.0209
Epoch 11/20
252/252 [==============================] - 0s 470us/step - loss: 0.0197
Epoch 12/20
252/252 [==============================] - 0s 460us/step - loss: 0.0185
E

In [18]:

from sklearn.utils import shuffle
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

n_inputs = len(assets)
n_core = N_COMPONENTS
n_outputs = n_inputs

initializer = tf.initializers.glorot_normal()
w1 = tf.Variable(initializer([n_inputs, n_core]))
w2 = tf.transpose(w1)
b1 = tf.Variable(tf.zeros([n_core]))
b2 = tf.Variable(tf.zeros([n_outputs]))

Instructions for updating:
non-resource variables are not supported in the long term


In [19]:

# Building the encoder
def encoder(x):
    return tf.nn.sigmoid(tf.add(tf.matmul(x, w1), b1))

# Building the decoder
def decoder(x):
    return tf.nn.sigmoid(tf.add(tf.matmul(x, w2), b2))

In [20]:
X = tf.placeholder("float", [None, n_inputs])
Y = tf.placeholder("float", [None, n_inputs])
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
# Prediction
y_pred = decoder_op
y_true = X


# Training parameters
lr = 0.01
epochs = 20
batch_size = 1


mse = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.AdamOptimizer(lr).minimize(mse)

# Start Training
# Start a new TF session
with tf.Session() as sess:
    # Initialize the network
    sess.run(tf.global_variables_initializer())

    # Training
    for i in range(epochs):
        X_train1 = shuffle(X_train)
        for j in range(X_train.shape[0] // batch_size):
            batch_y = X_train1[j * batch_size:j * batch_size + batch_size, :]
            batch_x = X_train1[j * batch_size:j * batch_size + batch_size, :]
            _, loss_value = sess.run([optimizer, mse], feed_dict={X: batch_x, Y: batch_y})

        # Display loss
        print('Epoch: %i -> Loss: %f' % (i, loss_value))

    # Make predictions
    y_pred_AE_tf = sess.run(decoder_op, feed_dict={X: X_train, Y: X_train})
    print('Test Error: %f' % tf.losses.mean_squared_error(X_train, y_pred_AE_tf).eval())

Epoch: 0 -> Loss: 0.001994
Epoch: 1 -> Loss: 0.003119
Epoch: 2 -> Loss: 0.001483
Epoch: 3 -> Loss: 0.004482
Epoch: 4 -> Loss: 0.001497
Epoch: 5 -> Loss: 0.000779
Epoch: 6 -> Loss: 0.000213
Epoch: 7 -> Loss: 0.001107
Epoch: 8 -> Loss: 0.000609
Epoch: 9 -> Loss: 0.000599
Epoch: 10 -> Loss: 0.000247
Epoch: 11 -> Loss: 0.000061
Epoch: 12 -> Loss: 0.004134
Epoch: 13 -> Loss: 0.000614
Epoch: 14 -> Loss: 0.000257
Epoch: 15 -> Loss: 0.000077
Epoch: 16 -> Loss: 0.000862
Epoch: 17 -> Loss: 0.000181
Epoch: 18 -> Loss: 0.000083
Epoch: 19 -> Loss: 0.000160
Test Error: 0.000390
